# Planning Notebook

## TODO

- Add '+': track

## Interface

Here's my idea for the interface. 

        root_container = HSplit(
            [
                menu_container,  # menu items
                HorizontalLine(),
                display_area,  # was log_area - only for display
                input_container,
                HorizontalLine(),
                message_window  
                ]
            )
        layout = Layout(root_container)

- The default display for the text area is a list of the trackers. E.g.

        a. Fill bird feeders
        b. Fill water dispenser
        ... 

    There would be a dictionary that maps the tracker labels, a. b. ..., to the corresponding tracker ID.

- The menu_container would be a list of menu items
    
        a)dd  d)elete  e)dit ... 
    
    Some of these, e.g.  d)elete, e)dit would require a tracker ID and some would not, e.g., a)dd.

- My idea is that the menu_mode would be true to begin with and in this mode the key bindings would bind "a" to "add" and "d" to "delete" and so forth. When a menu item is selected that requires an ID, "action" would store the selected menu item, menu_mode would change to False and select_mode would change to True. In select_mode the key bindings would bind "a" to "select_a" and "b" to "select_b" and so forth. "select_a" would set the selected ID to that associated with "a", and so forth. At this point the "action" could be executed using the selected ID.

What do you think?


## Birdseye View

*etm* is a collection of Item instances together with a ItemManager that provides an interface for the collection of items. Each item instance has a dictionary that contains all the attributes of the item and a collection of methods for interacting with the item. 

* from_entry_to_item: convert a string as it is being entered by the user to the item dictionary. 
* from_item_to_entry: convert the item dictionary to an entry string for editing by the user

## Tree Structure

- items (path tuple), (column tuple)
    - (A, ), a1 
    - (A, B), (ab1)
    - (A, B), (ab2)
    - (A, B, C), (abc1)
    - (A, B, C), (abc2)
    - (A, B, D), (abd1)

Tree 1
    A

Tree 2
    A
        a1
        B

Tree 3
    A
        a1
        B
            ab1
            ab2
            C
            D

Tree 4
    A
        a1
        B
            ab1
            ab2
            C
                abc1
                abc2
            D
                abd1



## 2024-07-21

## 2024-07-22

- Integrate @s (including tzinfo) into Repeat
- Work on process_entry for Item
    - called when buffer cursor position changes or when buffer text changes
    - status info for current entry
        - missing or invalid item type -> give list of item types
        - any token keys that are not allowed for item type?
        - any token key duplications that are not allowed?
        - any required token keys that are missing?
        - any invalid token values?
    - for the active token
        - no value -> give token description
        - invalid value -> give error message
        - partial value ->  feedback about how to complete
        - possibly complete value -> show current interpretation

## 2024-07-23

- @s and @r
    - now() is the default for dtstart for @r so @s can be omitted
    - with @+ but without @r, 
        - @s must be added to @+ to be included - probably best
        - or the value of @s could be added to @+ and @s could be omitted
        - or a dummy rrule could be created with a frequency of DAILY and a count of 1 and with dtstart = value of @s - worst
    - combinatons - subsets (s, r, +)
        - s
        - r: uses default now for dtstart in each @r 
        - +: 
        - s, r: use @s for dtstart in each @r
        - s, +: prepend @s to @+
        - r, +: uses now for dtstart in each @r
        - s, r, +: uses @s for dtstart in each @r



## Classes


### Item

If the instance is created from the tinydb, then it will have a doc_id attribute and will be a dictionary with keys for itemtype, summary, s, 'r' and so forth.

To display or edit the item, methods will be needed to convert the item to an entry string and to convert the entry string back to an item. The latter method will need to handle partial and/or incorrect entry strings by providing feedback about issues and suggestions as needed.

Methods will be needed to provide rows for the views that depend upon the  the item, i.e, "done" rows for undated tasks, "journal" rows for journal entries and so forth. 

Rows attributable to a particular doc_id will need to be easily updated with the item corresponding to that doc_id is modified, or when the the relevant range for datetimes is changed. 

One option would be to have a dict[doc_id, dict[view, list[view_rows]]. Another would be a dict[view, dict[doc_id, list[view_rows]]].


### Repeat (possibly included in Item?)

If the instance is created from the tinydb, then it will have an 'r' attribute that contains a rruleset object.

If the instance is being created from user input, then it will have an 'r' attribute that contains an entry string that needs to be converted to a rruleset object.

To display or edit the rruleset, a method will be needed to convert the rruleset to an entry string.

methods needed:

- entry_to_rruleset
    - 
- rruleset_to_entry
- rruleset_to_details

rruleset methods needed:

- after(dt, inc=False)
    Returns the **first** recurrence after the given datetime instance. The inc keyword defines what happens if dt is an occurrence. With inc=True, if dt itself is an occurrence, it will be returned.

    Note: relevant for rrules = after(datetime.now(), inc=True)

- before(dt, inc=False)
    Returns the **last** recurrence before the given datetime instance. The inc keyword defines what happens if dt is an occurrence. With inc=True, if dt itself is an occurrence, it will be returned.

- between(after, before, inc=False, count=1)
    Returns all the occurrences of the rrule between after and before. The inc keyword defines what happens if after and/or before are themselves occurrences. With inc=True, they will be included in the list, if they are found in the recurrence set.



- xafter(dt, count=None, inc=False)
    Generator which yields up to count recurrences after the given datetime instance, equivalent to after.

    Parameters:
    
    - dt – The datetime at which to start generating recurrences.

    - count – The maximum number of recurrences to generate. If None (default), dates are generated until the recurrence rule is exhausted.

    - inc – If dt is an instance of the rule and inc is True, it is included in the output.

    Yields a sequence of datetime objects.



### ItemManager (aka DataView)

from ChatGPT:

class AttrDict(dict):
    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(f"No such attribute: {name}")

    def __setattr__(self, name, value):
        self[name] = value

# Example usage
d = AttrDict()
d.prop = 'value'
print(d.prop)  # Output: value
print(d['prop'])  # Output: value


In [3]:
# NOTE: Item class with get_views_and_rows and ItemManager class
from collections import defaultdict
from datetime import date, timedelta
import textwrap
import shutil

def wrap(txt, indent=3, width=shutil.get_terminal_size()[0] - 1):
    """
    Wrap text to terminal width using indent spaces before each line.
    >>> txt = "Now is the time for all good men to come to the aid of their country. " * 5
    >>> res = wrap(txt, 4, 60)
    >>> print(res)
    Now is the time for all good men to come to the aid of
        their country. Now is the time for all good men to
        come to the aid of their country. Now is the time
        for all good men to come to the aid of their
        country. Now is the time for all good men to come
        to the aid of their country. Now is the time for
        all good men to come to the aid of their country.
    """
    para = [x.rstrip() for x in txt.split('\n')]
    tmp = []
    first = True
    for p in para:
        if first:
            initial_indent = ''
            first = False
        else:
            initial_indent = ' ' * indent
        tmp.append(
            textwrap.fill(
                p,
                initial_indent=initial_indent,
                subsequent_indent=' ' * indent,
                width=width - indent - 1,
            )
        )
    return '\n'.join(tmp)

def print_wrapped(txt):
    print(wrap(txt))

class Item:
    def __init__(self, doc_id, name, item_type, item_date=None, completion_date=None):
        self.doc_id = doc_id
        self.item_type = item_type
        self.name = name
        self.item_date = item_date
        self.completion_date = completion_date
        self.is_completed = (
            self.item_type == 'task' and
            self.completion_date is not None
            )
        # Add other properties as needed
        # Add other properties as needed

    def get_views_and_rows(self):
        views_and_rows = {}

        # Example logic for adding to agenda view if there is an event date
        if self.item_date:
            YrWk = self.item_date.isocalendar()[:2]
        elif self.completion_date:
            YrWk = self.completion_date.isocalendar()[:2]
        else:
            YrWk = None

        if self.item_type == 'event':
            if self.item_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.item_date, self.name)]

        # Example logic for adding to tasks view if there is a due date and it's not completed
        # Example logic for adding to completed view if it is completed
        elif self.item_type == 'task':
            if self.completion_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.completion_date, self.name)]
            elif self.item_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.item_date, self.name)]
            else:
                views_and_rows['tasks'] = [(None, self.name)]


        # Add other views and logic as needed based on the properties of the item
        # print(f"{views_and_rows = }")
        return views_and_rows


from collections import defaultdict
from datetime import date

class ItemManager:
    def __init__(self):
        self.doc_view_data = {}  # Primary structure: dict[doc_id, dict[view, list[row]]]
        self.view_doc_data = defaultdict(lambda: defaultdict(list))  # Secondary index: dict[view, dict[doc_id, list[row]])
        self.view_cache = {}  # Cache for views
        self.doc_view_contribution = defaultdict(set)  # Tracks views each doc_id contributes to

    def add_or_update_reminder(self, item):
        doc_id = item.doc_id
        new_views_and_rows = item.get_views_and_rows()

        # Invalidate cache for views that will be affected by this doc_id
        self.invalidate_cache_for_doc(doc_id)

        # Update the primary structure
        self.doc_view_data[doc_id] = new_views_and_rows

        # Update the secondary index
        for view, rows in new_views_and_rows.items():
            self.view_doc_data[view][doc_id] = rows
            self.doc_view_contribution[doc_id].add(view)

    def get_view_data(self, view):
        # Check if the view is in the cache
        if view in self.view_cache:
            return self.view_cache[view]

        # Retrieve data for a specific view
        view_data = dict(self.view_doc_data[view])

        # Cache the view data
        self.view_cache[view] = view_data
        return view_data

    def get_reminder_data(self, doc_id):
        # Retrieve data for a specific reminder
        return self.doc_view_data.get(doc_id, {})

    def remove_reminder(self, doc_id):
        # Invalidate cache for views that will be affected by this doc_id
        self.invalidate_cache_for_doc(doc_id)

        # Remove reminder from primary structure
        if doc_id in self.doc_view_data:
            views_and_rows = self.doc_view_data.pop(doc_id)
            # Remove from secondary index
            for view in views_and_rows:
                if doc_id in self.view_doc_data[view]:
                    del self.view_doc_data[view][doc_id]

            # Remove doc_id from contribution tracking
            if doc_id in self.doc_view_contribution:
                del self.doc_view_contribution[doc_id]

    def invalidate_cache_for_doc(self, doc_id):
        # Invalidate cache entries for views affected by this doc_id
        if doc_id in self.doc_view_contribution:
            for view in self.doc_view_contribution[doc_id]:
                if view in self.view_cache:
                    del self.view_cache[view]

# Example usage
manager = ItemManager()

# Create some reminders (Item instances)
item1 = Item(1, "Independence Day", "event", item_date=date(2024, 7, 4))
item2 = Item(2, "Dated", "task", item_date=date(2024, 7, 10))
item3 = Item(3, "UnDated", "task")
item4 = Item(4, "Undated and Completed", "task", completion_date=date(2024, 7, 10))
item5 = Item(5, "Dated and Completed", "task", item_date=date(2024, 7, 4), completion_date=date(2024, 7, 6))

# Add or update reminders
manager.add_or_update_reminder(item1)
manager.add_or_update_reminder(item2)
manager.add_or_update_reminder(item3)
manager.add_or_update_reminder(item4)
manager.add_or_update_reminder(item5)

# Query view data
view_data_agenda = manager.get_view_data(((2024, 27), "agenda"))
print_wrapped(f"Agenda View Data for 2024-27:\n{view_data_agenda}")
view_data_agenda = manager.get_view_data(((2024, 28), "agenda"))
print_wrapped(f"Agenda View Data for 2024-28:\n{view_data_agenda}")

view_data_tasks = manager.get_view_data("tasks")
print_wrapped(f"Tasks View Data:\n{view_data_tasks}")

# Query reminder data
reminder_data_1 = manager.get_reminder_data(1)
print_wrapped(f"Reminder 1 Data:\n{reminder_data_1}")

print_wrapped(f"view_cache:\n{manager.view_cache}")

# Remove a reminder
manager.remove_reminder(5)
print("After removal of item 5")
view_data_agenda = manager.get_view_data(((2024, 27), "agenda"))
print_wrapped(f"Agenda View Data for 2024-27:\n {view_data_agenda}")
view_data_agenda = manager.get_view_data(((2024, 28), "agenda"))
print_wrapped(f"Agenda View Data for 2024-28:\n{view_data_agenda}")

Agenda View Data for 2024-27:
   {1: [(datetime.date(2024, 7, 4), 'Independence Day')], 5:
   [(datetime.date(2024, 7, 6), 'Dated and Completed')]}
Agenda View Data for 2024-28:
   {2: [(datetime.date(2024, 7, 10), 'Dated')], 4: [(datetime.date(2024, 7,
   10), 'Undated and Completed')]}
Tasks View Data:
   {3: [(None, 'UnDated')]}
Reminder 1 Data:
   {((2024, 27), 'agenda'): [(datetime.date(2024, 7, 4), 'Independence
   Day')]}
view_cache:
   {((2024, 27), 'agenda'): {1: [(datetime.date(2024, 7, 4), 'Independence
   Day')], 5: [(datetime.date(2024, 7, 6), 'Dated and Completed')]},
   ((2024, 28), 'agenda'): {2: [(datetime.date(2024, 7, 10), 'Dated')], 4:
   [(datetime.date(2024, 7, 10), 'Undated and Completed')]}, 'tasks': {3:
   [(None, 'UnDated')]}}
After removal of item 5
Agenda View Data for 2024-27:
    {1: [(datetime.date(2024, 7, 4), 'Independence Day')]}
Agenda View Data for 2024-28:
   {2: [(datetime.date(2024, 7, 10), 'Dated')], 4: [(datetime.date(2024, 7,
   10), 'Undated a